<div class="alert alert-block" style="border: 1px solid #000000FF;background-color:#E3F2FD;">
<font size="4em" style="font-weight:bold;color:#000000FF;">코로나 데이터 전처리 & 하나의 csv로 통합</font><br>

##### 확진자 수 추이를 볼 수 있는 그래프를 그리기 위한 **전처리 과정**
</div>

In [158]:
import pandas as pd

PATH = 'COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/'
apr_doc = pd.read_csv(PATH+'04-01-2020.csv',encoding='utf-8-sig')
doc.head()

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,Anhui,Mainland China,1/22/2020 17:00,1.0,NaN,NaN
1,Beijing,Mainland China,1/22/2020 17:00,14.0,NaN,NaN
2,Chongqing,Mainland China,1/22/2020 17:00,6.0,NaN,NaN
3,Fujian,Mainland China,1/22/2020 17:00,1.0,NaN,NaN
4,Gansu,Mainland China,1/22/2020 17:00,NaN,NaN,NaN


In [159]:
mar_doc = pd.read_csv(PATH+'03-01-2020.csv',encoding='utf-8-sig')
mar_doc.head()

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude
0,Hubei,Mainland China,2020-03-01T10:13:19,66907,2761,31536,30.9756,112.2707
1,NaN,South Korea,2020-03-01T23:43:03,3736,17,30,36.0000,128.0000
2,NaN,Italy,2020-03-01T23:23:02,1694,34,83,43.0000,12.0000
3,Guangdong,Mainland China,2020-03-01T14:13:18,1349,7,1016,23.3417,113.4244
4,Henan,Mainland China,2020-03-01T14:13:18,1272,22,1198,33.8820,113.6140


<div class="alert alert-block" style="border: 1px solid #000000FF;background-color:#E3F2FD;">
<font size="3em" style="font-weight:bold;color:#000000FF;">컬럼명 통합</font><br>

##### 컬럼이 3월까지 **Country/Region, Province/State** 이었다가 **Country_Region, Province_State**으로 바뀌었다.

##### Country_Region, Province_State으로 컬럼명을 통합하자.
</div>

In [161]:
doc = pd.read_csv(PATH+'01-22-2020.csv',encoding='utf-8-sig')

doc.head()

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,Anhui,Mainland China,1/22/2020 17:00,1.0,NaN,NaN
1,Beijing,Mainland China,1/22/2020 17:00,14.0,NaN,NaN
2,Chongqing,Mainland China,1/22/2020 17:00,6.0,NaN,NaN
3,Fujian,Mainland China,1/22/2020 17:00,1.0,NaN,NaN
4,Gansu,Mainland China,1/22/2020 17:00,NaN,NaN,NaN


In [162]:
doc = pd.read_csv(PATH+'01-22-2020.csv',encoding='utf-8-sig')
try:
    doc[['Province_State', 'Country_Region', 'Confirmed']]
except:
    target_doc = doc[['Province/State', 'Country/Region', 'Confirmed']]
    target_doc.columns = ['Province_State', 'Country_Region', 'Confirmed']

target_doc.head()

,Province_State,Country_Region,Confirmed
0,Anhui,Mainland China,1.0
1,Beijing,Mainland China,14.0
2,Chongqing,Mainland China,6.0
3,Fujian,Mainland China,1.0
4,Gansu,Mainland China,NaN


<div class="alert alert-block" style="border: 1px solid #000000FF;background-color:#E3F2FD;">
<font size="3em" style="font-weight:bold;color:#000000FF;">Nan 제거 후 Confirmed타입 변경</font><br>

##### float타입을 int로 바꾸자. 이를 위해 nan값을 가지고 있는 row를 제거하자.
</div>

In [164]:
doc = pd.read_csv(PATH+'01-22-2020.csv',encoding='utf-8-sig')
try:
    doc[['Province_State', 'Country_Region', 'Confirmed']]
except:
    target_doc = doc[['Province/State', 'Country/Region', 'Confirmed']]
    target_doc.columns = ['Province_State', 'Country_Region', 'Confirmed']

result_doc = target_doc.dropna(subset=['Confirmed'])
result_doc = result_doc.astype({'Confirmed':'int64'})

result_doc.head()

,Province_State,Country_Region,Confirmed
0,Anhui,Mainland China,1
1,Beijing,Mainland China,14
2,Chongqing,Mainland China,6
3,Fujian,Mainland China,1
5,Guangdong,Mainland China,26


<div class="alert alert-block" style="border: 1px solid #000000FF;background-color:#E3F2FD;">
<font size="3em" style="font-weight:bold;color:#000000FF;">국가명 표준으로 변경</font><br>
    
---

##### 현재 국가명은 공식 이름이 아닌 다양한 방법으로 적혀져 있다. 
##### ex) China, Mainland China, Macau 등등 
<br>

##### 따라서 이를 apply함수를 이용해서 공식 이름으로 통합하고 
##### 국가 공식 명칭에 대응하는 iso(국가 나타내는 대문자 두글자)테이블과 merge를 하자

</div>

In [166]:
import json

with open('COVID-19-master/csse_covid_19_data/country_convert.json','r',encoding='utf-8-sig') as json_file:
    json_data = json.load(json_file)
    print(json_data)

{'Mainland China': 'China', 'Macau': 'China', 'South Korea': 'Korea, South', 'Aruba': 'Netherlands', ' Azerbaijan': 'Azerbaijan', 'Bahamas, The': 'Bahamas', 'Cape Verde': 'Cabo Verde', 'Cayman Islands': 'United Kingdom', 'Channel Islands': 'United Kingdom', 'Curacao': 'Netherlands', 'Czech Republic': 'Czechia', 'East Timor': 'Timor-Leste', 'Faroe Islands': 'Denmark', 'French Guiana': 'France', 'Gambia, The': 'Gambia', 'Gibraltar': 'United Kingdom', 'Greenland': 'Denmark', 'Guadeloupe': 'France', 'Guam': 'US', 'Guernsey': 'US', 'Hong Kong': 'China', 'Hong Kong SAR': 'China', 'Iran (Islamic Republic of)': 'Iran', 'Ivory Coast': "Cote d'Ivoire", 'Jersey': 'US', 'Macao SAR': 'China', 'Martinique': 'France', 'Mayotte': 'France', 'North Ireland': 'United Kingdom', 'Palestine': 'West Bank and Gaza', 'Puerto Rico': 'US', 'Republic of Ireland': 'Ireland', 'Republic of Korea': 'Korea, South', 'Republic of Moldova': 'Moldova', 'Republic of the Congo': 'Congo (Brazzaville)', 'Reunion': 'France', '

In [167]:
def func(row):
    if row['Country_Region'] in json_data:
        row['Country_Region'] = json_data[row['Country_Region']]
    return row

In [168]:
# apply함수 : 데이터 프레임을 한 줄씩 시리즈로 나누고 그 시리즈를 func에 전달함.
# 시리즈 : 일차원 배열 
#          - 인덱스와, 일차원 배열, name속성(주로 컬럼명 담김)이 존재한다.

result_doc = result_doc.apply(func,axis=1)
result_doc.head()

,Province_State,Country_Region,Confirmed
0,Anhui,China,1
1,Beijing,China,14
2,Chongqing,China,6
3,Fujian,China,1
5,Guangdong,China,26


<div class="alert alert-block" style="border: 1px solid #000000FF;background-color:#E3F2FD;">
<font size="3em" style="font-weight:bold;color:#000000FF;">문자열 변경하기</font><br>
    
---

##### 파일 이름(날짜)를 가져와 적절히 변경을 해 컬럼으로 사용할 것이다.
##### 이를 문자열 함수를 사용해 해결해보자.

</div>

In [184]:

file_name = '01-23-2020.csv' #예시
column_name = file_name.split('.')[0].lstrip('0').replace('-','/')
column_name


'1/23/2020'

In [191]:
# 컬럼 이름을 날짜로 수정하자.
result_doc.columns = ['Province_State', 'Country_Region', column_name]
result_doc.head()

,Province_State,Country_Region,1/23/2020
0,Anhui,China,1
1,Beijing,China,14
2,Chongqing,China,6
3,Fujian,China,1
5,Guangdong,China,26


<div class="alert alert-block" style="border: 1px solid #000000FF;background-color:#E3F2FD;">
<font size="3em" style="font-weight:bold;color:#000000FF;">중복 데이터 합치기</font><br>
    
---

##### 국가 이름을 json을 통해 표준 이름으로 합친 결과 같은 국가가 여러 row에 존재하게 되었다. 
##### 이런 row들을 통합하자.

</div>

In [197]:
# 먼저 수 데이터가 존재하는 컬럼 + groupby 컬럼 따로 추출
extracted_df = result_doc[['Country_Region','1/23/2020']]

extracted_df = extracted_df.groupby('Country_Region').sum()
extracted_df.head()


,1/23/2020
Country_Region,
Antarctica,0
China,548
Japan,2
Kiribati,0
"Korea, North",0


In [205]:
#만약 문자열 있는데 groupby 컬럼 아닌데 포함 하면?
wrong_df = result_doc.groupby('Country_Region').sum()
wrong_df.head()

,Province_State,1/23/2020
Country_Region,,
Antarctica,0,0
China,AnhuiBeijingChongqingFujianGuangdongGuangxiGui...,548
Japan,0,2
Kiribati,0,0
"Korea, North",0,0


<div class="alert alert-block" style="border: 1px solid #000000FF;background-color:#E3F2FD;">
<font size="4em" style="font-weight:bold;color:#000000FF;">지금까지 과정 함수로 구조화 </font><br>

</div>

In [316]:
import json
import pandas as pd


with open('COVID-19-master/csse_covid_19_data/country_convert.json','r', encoding='utf-8-sig') as json_file:
    json_data = json.load(json_file)


def correct_country_name(row):
    if row['Country_Region'] in json_data:
        return json_data[row['Country_Region']]
    return row['Country_Region']


def preprocess_df(csv_file_name):
   csv_file = pd.read_csv(PATH+csv_file_name, encoding='utf-8-sig')
   try:
        extracted_df = csv_file[['Country_Region','Confirmed']]
   except: 
       extracted_df = csv_file[['Country/Region', 'Confirmed']]
       extracted_df.columns = ['Country_Region','Confirmed']
   extracted_df = extracted_df.dropna(subset=['Confirmed'])
   extracted_df['Country_Region'] = extracted_df.apply(correct_country_name,axis=1)
   extracted_df = extracted_df.astype({'Confirmed':'int64'})
   extracted_df = extracted_df.groupby('Country_Region').sum() # 해당 컬럼 index로 감
   csv_file_name = csv_file_name.split('.')[0].lstrip('0').replace('-','/')
   extracted_df.columns = [csv_file_name]
   return extracted_df

In [264]:
## 테스트 ##

df1 = preprocess_df("01-22-2020.csv")
df1.head()


,1/22/2020
Country_Region,
Antarctica,0
China,548
Japan,2
Kiribati,0
"Korea, North",0


<div class="alert alert-block" style="border: 1px solid #000000FF;background-color:#E3F2FD;">
<font size="4em" style="font-weight:bold;color:#000000FF;">데이터 프레임 합치기</font><br>
    
---


#### 인덱스로 합치는 함수

- ##### merge(df1, df2, how='outer', left_index=True, right_index=True)

</div>

In [268]:
doc2 = '04-01-2020.csv'
df2 = preprocess_df(doc2)

merged_df = pd.merge(df1,df2,how='outer',left_index=True,right_index=True)
merged_df.head()

,1/22/2020,4/01/2020
Country_Region,,
Afghanistan,NaN,192
Albania,NaN,259
Algeria,NaN,847
Andorra,NaN,390
Angola,NaN,8


In [272]:
##Nan 값을 0으로 바꾸기
merged_df = merged_df.fillna(0)
merged_df.head()

,1/22/2020,4/01/2020
Country_Region,,
Afghanistan,0.0,192
Albania,0.0,259
Algeria,0.0,847
Andorra,0.0,390
Angola,0.0,8


<div class="alert alert-block" style="border: 1px solid #000000FF;background-color:#E3F2FD;">
<font size="4em" style="font-weight:bold;color:#000000FF;">디렉토리 안에 있는 파일이름 다 가져오기</font><br>
    
---


#### os.listdir(PATH)


</div>

In [287]:
import os

file_name_list = os.listdir(PATH)
valid_storage = list()

for i in file_name_list:
    if(i.split('.')[-1]=='csv'):
        valid_storage.append(i)

j=0

for i in valid_storage:
    if(j==5):
        break
    else:
        print(valid_storage[j])
        j=j+1


01-21-2022.csv
02-26-2020.csv
01-20-2022.csv
02-27-2020.csv
07-04-2021.csv


<div class="alert alert-block" style="border: 1px solid #000000FF;background-color:#E3F2FD;">
<font size="4em" style="font-weight:bold;color:#000000FF;">날짜가 들어가 있는 문자열 날짜순으로 정렬하기</font><br>
    
---


##### ```from datetime import datetime```


</div>

In [303]:
from datetime import datetime

valid_storage.sort(key=lambda x: datetime.strptime(x, '%m-%d-%Y.csv'))


<div class="alert alert-block" style="border: 1px solid #000000FF;background-color:#E3F2FD;">
<font size="5em" style="font-weight:bold;color:#000000FF;">최종 코드</font><br>


</div>

In [324]:
import os
from datetime import datetime

def get_final_dataframe(PATH):
    file_list, csv_list = os.listdir(PATH), list()
    first_doc = True
    for file in file_list:
        if file.split('.')[-1] == 'csv':
            csv_list.append(file)
            
    csv_list.sort(key=lambda x:datetime.strptime(x,'%m-%d-%Y.csv'))

    for file in csv_list:
        preprocessed_doc = preprocess_df(file)
        if first_doc:
            final_doc, first_doc = preprocessed_doc, False
        else:
            # 이제부터는 아우터 머지
            final_doc = pd.merge(final_doc,preprocessed_doc,how='outer',left_index=True,right_index=True)

    final_doc = final_doc.fillna(0)
    return final_doc

In [326]:
PATH = 'COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/'
doc = get_final_dataframe(PATH)
doc


,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,1/31/2020,...,2/28/2023,3/01/2023,3/02/2023,3/03/2023,3/04/2023,3/05/2023,3/06/2023,3/07/2023,3/08/2023,3/09/2023
Country_Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,209322.0,209340.0,209358.0,209362.0,209369.0,209390.0,209406.0,209436.0,209451.0,209451.0
Albania,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,334391.0,334408.0,334408.0,334427.0,334427.0,334427.0,334427.0,334427.0,334443.0,334457.0
Algeria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,271441.0,271448.0,271463.0,271469.0,271469.0,271477.0,271477.0,271490.0,271494.0,271496.0
Andorra,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,47866.0,47875.0,47875.0,47875.0,47875.0,47875.0,47875.0,47875.0,47890.0,47890.0
Angola,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,105255.0,105277.0,105277.0,105277.0,105277.0,105277.0,105277.0,105277.0,105288.0,105288.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
West Bank and Gaza,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,703228.0,703228.0,703228.0,703228.0,703228.0,703228.0,703228.0,703228.0,703228.0,703228.0
Winter Olympics 2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,535.0,535.0,535.0,535.0,535.0,535.0,535.0,535.0,535.0,535.0
Yemen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11945.0,11945.0,11945.0,11945.0,11945.0,11945.0,11945.0,11945.0,11945.0,11945.0


In [330]:
doc = doc.fillna(0)
doc = doc.astype('int64')
doc

,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,1/31/2020,...,2/28/2023,3/01/2023,3/02/2023,3/03/2023,3/04/2023,3/05/2023,3/06/2023,3/07/2023,3/08/2023,3/09/2023
Country_Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,209322,209340,209358,209362,209369,209390,209406,209436,209451,209451
Albania,0,0,0,0,0,0,0,0,0,0,...,334391,334408,334408,334427,334427,334427,334427,334427,334443,334457
Algeria,0,0,0,0,0,0,0,0,0,0,...,271441,271448,271463,271469,271469,271477,271477,271490,271494,271496
Andorra,0,0,0,0,0,0,0,0,0,0,...,47866,47875,47875,47875,47875,47875,47875,47875,47890,47890
Angola,0,0,0,0,0,0,0,0,0,0,...,105255,105277,105277,105277,105277,105277,105277,105277,105288,105288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
West Bank and Gaza,0,0,0,0,0,0,0,0,0,0,...,703228,703228,703228,703228,703228,703228,703228,703228,703228,703228
Winter Olympics 2022,0,0,0,0,0,0,0,0,0,0,...,535,535,535,535,535,535,535,535,535,535
Yemen,0,0,0,0,0,0,0,0,0,0,...,11945,11945,11945,11945,11945,11945,11945,11945,11945,11945


In [334]:
## 저장 
doc.to_csv("COVID-19-master/giyeon_final_df.csv")